In [21]:
using Pkg
Pkg.status()
Pkg.update()

      Status `C:\Users\kappe\.julia\environments\v1.7\Project.toml`
  [9961bab8] Cbc v1.0.1
  [ae04f764] ConditionalJuMP v0.1.0
  [c04bee98] ExcelReaders v0.11.0
  [42e2da0e] Grisu v1.0.2
  [87dc4568] HiGHS v1.1.4
  [7073ff75] IJulia v1.23.3
  [4076af6c] JuMP v0.18.6
  [fdbf4ff8] XLSX v0.7.10


    Updating registry at `C:\Users\kappe\.julia\registries\General.toml`
  No Changes to `C:\Users\kappe\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\kappe\.julia\environments\v1.7\Manifest.toml`


In [22]:
using HiGHS
using XLSX

In [23]:
using JuMP

┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1423
ERROR: LoadError: UndefVarError: Grisu not defined
Stacktrace:
  [1] getproperty(x::Module, f::Symbol)
    @ Base .\Base.jl:35
  [2] top-level scope
    @ C:\Users\kappe\.julia\packages\JuMP\I7whV\src\writers.jl:6
  [3] include(mod::Module, _path::String)
    @ Base .\Base.jl:418
  [4] include(x::String)
    @ JuMP C:\Users\kappe\.julia\packages\JuMP\I7whV\src\JuMP.jl:11
  [5] top-level scope
    @ C:\Users\kappe\.julia\packages\JuMP\I7whV\src\JuMP.jl:947
  [6] include
    @ .\Base.jl:418 [inlined]
  [7] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt64}}, source::Nothing)
    @ Base .\loading.jl:1318
  [8] top-level scope
    @ none:1
  [9] eval
    @ .\boot.jl:373 [inlined]
 [10] eval(x::Expr)
    @ Base.MainInclude .\client.jl:453
 [11] top-level scope
    

LoadError: Failed to precompile JuMP [4076af6c-e467-56ae-b986-b466b2749572] to C:\Users\kappe\.julia\compiled\v1.7\JuMP\jl_978B.tmp.

In [18]:
data = XLSX.readxlsx("Stackup_Overlay_error.xlsx")
t = data["Layer 1"]["A2:A201"]
L1 = data["Layer 1"]["B2:B201"]
L2 = data["Layer 2"]["B2:B201"]
L3 = data["Layer 3"]["B2:B201"]
L4 = data["Layer 4"]["B2:B201"]
L5 = data["Layer 5"]["B2:B201"]
u = [L1,L2,L3,L4,L5]
#accum = 1
#u = [0,0,0,0,0]
#for sheetname in XLSX.sheetnames(data)
#    u[accum] = data[sheetname]["B2:B201"]
    #append!(u, data[sheetname]["B2:B201"])
#    accum += 1
#end
k = [[0,1,2,3,4,5]]
u[2][3]

-0.2447733783382714

Questions:

Do I have to define the variable before I use it in the constraints? How do I do it? ( also x)

should the three equations be constraints or should they be in the objective equation?



In [19]:
m = Model(HiGHS.Optimizer)

@variable(m, a[k = 1:5])
@variable(m, b[k = 1:5])
@variable(m, c[k = 1:5])
@variable(m,0 <= gamma[k = 1:5] <= 1)

s[k][t] = s[k - 1][t] + u[k][t + 1]


@constraint(m, s[k = 0][t] = 0)
@constraint(m ,x[k = 1:5][0] = 0)
@constraint(m, u[k = 1:5][0] = 0)
@constraint(m, sum(gamma[k]) == 1)
@constranit(m, x[k][t] = a[k] * x[k][t - 1] + b[k] * u[k][t])

@objective(m, Min, gamma[k][t] * abs(s[k-1][t] + c[k] * (a[k] * x[k][t-1] + b[k] * u[k][t]))^2)
    


LoadError: UndefVarError: Model not defined

In [ ]:
m = Model(HiGHS.Optimizer)

@variable(m, a[k = 1:5])
@variable(m, b[k = 1:5])
@variable(m, c[k = 1:5])
@variable(m,0 <= gamma[k = 1:5] <= 1)

@constraint(m, s[k = 0][t] == 0)
@constraint(m ,x[k = 1:5][0] = 0)
@constraint(m, u[k = 1:5][0] = 0)
@constraint(m, sum(gamma[k]) == 1)
@constranit(m, x[k][t] = a[k] * x[k][t - 1] + b[k] * u[k][t])
@constranit(m, u[k][t+1] = c[k] * x[k][t])
@constranit(m, s[k][t] = s[k - 1][t] + u[k][t + 1]
    
@objective(m, Min, sum(gamma[k][t] * abs(s[k][t])^2))
    


In [ ]:
radler = Model(HiGHS.Optimizer)
#set_optimizer_attribute(model, "logLevel", 1)

@variable(radler, p >= 0, Int)
@variable(radler, n >= 0, Int)
@variable(radler, d >= 0, Int)
@variable(radler, q >= 0, Int)


@constraint(radler, 1*p + 5*n + 10*d + 25*q == 99)

@objective(radler, Max, 2.5*p + 5*n + 2.268*d + 5.670*q )

optimize!(radler)
#print(m)
#solution_summary(model)

In [ ]:
print(value(p))
print(value(n))
print(value(d))
print(value(q))
